In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import time
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
import os

def train_network(net = None, train_set = None, val_set = None, device = None,
epochs = 10, bs = 20, optimizer = None, criterion = None):  # outdir = None, file_prefix = None):

    train_loader = DataLoader(train_set, batch_size=bs, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=bs, shuffle=True)

    net = net.to(device)

    tr_losses = []
    val_losses = []
    tr_accs = []
    val_accs = []

    for epoch in range(epochs):
        t1 = time.time()
        net.train()
        tr_loss = 0

        y_trues = []
        y_preds = []

        for i, sampled_batch in enumerate(train_loader):

            t2 = time.time()

            data = sampled_batch['feature']
            y = sampled_batch['label'].squeeze()

            data = data.type(torch.FloatTensor)
            y = y.type(torch.LongTensor)

            data = data.to(device)
            y = y.to(device)

            optimizer.zero_grad()
            output = net(data)

            loss = criterion(output,y)

            loss.backward()
            optimizer.step()
            #print(net.ql1.weights.grad)
            tr_loss = tr_loss + loss.data.cpu().numpy()

            y_trues += y.cpu().numpy().tolist()
            y_preds += output.data.cpu().numpy().argmax(axis=1).tolist()

            print('batch({}):{:.4f}'.format(i,time.time()-t2))

        tr_acc = accuracy_score(y_trues, y_preds)
        tr_accs.append(tr_acc)
        tr_loss = tr_loss/(i+1)
        tr_losses.append(tr_loss)

        cnf = confusion_matrix(y_trues, y_preds)
        print(cnf)

        print('Epoch:{}, TR_Loss: {:.4f}, TR_Acc: {:.4f}'.format(epoch, tr_loss, tr_acc))

        net.eval()
        val_loss = 0

        y_trues = []
        y_preds = []

        for i, sampled_batch in enumerate(val_loader):
            data = sampled_batch['feature']
            y = sampled_batch['label'].squeeze()

            data = data.type(torch.FloatTensor)
            y = y.type(torch.LongTensor)

            data = data.to(device)
            y = y.to(device)

            with torch.no_grad():
                output = net(data)

            loss = criterion(output,y)
            val_loss = val_loss + loss.data.cpu().numpy()

            y_trues += y.cpu().numpy().tolist()
            y_preds += output.data.cpu().numpy().argmax(axis=1).tolist()

        val_acc = accuracy_score(y_trues, y_preds)
        val_accs.append(val_acc)
        val_loss = val_loss/(i+1)
        val_losses.append(val_loss)

        cnf = confusion_matrix(y_trues, y_preds)
        print(cnf)

        print('Epoch: {} VAL_Loss: {:.4f}, VAL_Acc: {:.4f}'.format(epoch, val_loss, val_acc))
        print('Time for Epoch ({}): {:.4f}'.format(epoch, time.time()-t1))

    #save model and results
    # os.makedirs(outdir, exist_ok = True)
    # torch.save(net.state_dict(), outdir + '/' + file_prefix + '_model')
    # np.save(outdir + '/' + file_prefix + '_training_loss.npy', tr_losses)
    # np.save(outdir + '/' + file_prefix + '_validation_loss.npy', val_losses)
    # np.save(outdir + '/' + file_prefix + '_training_accuracy.npy', tr_accs)
    # np.save(outdir + '/' + file_prefix + '_validation_accuracy.npy', val_accs)

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(0)

n_class = 3
n_features = 196

class Net(nn.Module):
    # define nn
    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Conv2d(1, 4, 2, stride = 2)
        # self.lr1 = nn.LeakyReLU(0.1)
        # self.ln1 = nn.LayerNorm(32, elementwise_affine=True)
        self.fc1 = nn.Linear(4*7*7, 6)
        self.fc2 = nn.Linear(6, 3)

    def forward(self, X):
        bs = X.shape[0]
        X = X.view(X.shape[0], 1, 14, 14)
        X = self.conv(X)
        X = F.relu(X)
        X = X.view(bs,-1)
        X = self.fc1(X)
        X = F.relu(X)
        X = self.fc2(X)
        return X

In [6]:
!pip install pennylane # This line installs PennyLane

# ipython-input-3-dd0c84b8a7e9
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import time
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
import os

# ... (Rest of the code in ipython-input-3-dd0c84b8a7e9 remains unchanged) ...

# ipython-input-4-dd0c84b8a7e9
import torch
import torch.nn as nn
import torch.nn.functional as F

# ... (Rest of the code in ipython-input-4-dd0c84b8a7e9 remains unchanged) ...

# ipython-input-5-dd0c84b8a7e9
import torch
import torch.nn as nn
import numpy as np
import pennylane as qml # This import should now work
from math import ceil
from math import pi

torch.manual_seed(0)

n_qubits = 4
n_layers = 1
n_class = 3
n_features = 196
image_x_y_dim = 14
kernel_size = n_qubits
stride = 2

dev = qml.device("default.qubit", wires=n_qubits)


def circuit(inputs, weights):
    var_per_qubit = int(len(inputs) / n_qubits) + 1
    encoding_gates = ['RZ', 'RY'] * ceil(var_per_qubit / 2)
    for qub in range(n_qubits):
        qml.Hadamard(wires=qub)
        for i in range(var_per_qubit):
            if (qub * var_per_qubit + i) < len(inputs):
                exec('qml.{}({}, wires = {})'.format(encoding_gates[i], inputs[qub * var_per_qubit + i], qub))
            else:  # load nothing
                pass

    for l in range(n_layers):
        for i in range(n_qubits):
            qml.CRZ(weights[l, i], wires=[i, (i + 1) % n_qubits])
            # qml.CNOT(wires = [i, (i + 1) % n_qubits])
        for j in range(n_qubits, 2 * n_qubits):
            qml.RY(weights[l, j], wires=j % n_qubits)

    _expectations = [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]
    return _expectations
    # return qml.expval(qml.PauliZ(0))


class Quanv2d(nn.Module):
    def __init__(self, kernel_size=None, stride=None):
        super(Quanv2d, self).__init__()
        weight_shapes = {"weights": (n_layers, 2 * n_qubits)}
        qnode = qml.QNode(circuit, dev, interface='torch', diff_method='best')
        self.ql1 = qml.qnn.TorchLayer(qnode, weight_shapes)
        self.kernel_size = kernel_size
        self.stride = stride

    def forward(self, X):
        assert len(X.shape) == 4
        bs = X.shape[0]
        XL = []
        for i in range(0, X.shape[2] - 2, stride):
            for j in range(0, X.shape[3] - 2, stride):
                XL.append(self.ql1(torch.flatten(X[:, :, i:i + kernel_size, j:j + kernel_size], start_dim=1)))
        X = torch.cat(XL, dim=1).view(bs,4,6,6)
        return X


class Net(nn.Module):
    # define nn
    def __init__(self):
        super(Net, self).__init__()
        self.ql1 = Quanv2d(kernel_size=kernel_size, stride=stride)
        self.conv1 = nn.Conv2d(4,16,3,stride=1)
        self.fc1 = nn.Linear(16*4*4, n_class * 2)
        self.lr1 = nn.LeakyReLU(0.1)
        self.fc2 = nn.Linear(n_class * 2, n_class)

    def forward(self, X):
        bs = X.shape[0]
        X = X.view(bs, 1, image_x_y_dim, image_x_y_dim)
        X = self.ql1(X)
        X = self.lr1(self.conv1(X))
        X = X.view(bs,-1)
        X = self.fc1(X)
        X = self.lr1(X)
        X = self.fc2(X)
        return X

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.0/930.0 kB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 73.5 MB/s eta 0:00:00


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pennylane as qml
from math import ceil
from math import pi

torch.manual_seed(0)

n_qubits = 4
n_layers = 1
n_class = 3
n_features = 196
image_x_y_dim = 14
kernel_size = n_qubits
stride = 2

dev = qml.device("default.qubit", wires=n_qubits)


def circuit(inputs, weights):
    var_per_qubit = int(len(inputs) / n_qubits) + 1
    encoding_gates = ['RZ', 'RY'] * ceil(var_per_qubit / 2)
    for qub in range(n_qubits):
        qml.Hadamard(wires=qub)
        for i in range(var_per_qubit):
            if (qub * var_per_qubit + i) < len(inputs):
                exec('qml.{}({}, wires = {})'.format(encoding_gates[i], inputs[qub * var_per_qubit + i], qub))
            else:  # load nothing
                pass

    for l in range(n_layers):
        for i in range(n_qubits):
            qml.CRZ(weights[l, i], wires=[i, (i + 1) % n_qubits])
            # qml.CNOT(wires = [i, (i + 1) % n_qubits])
        for j in range(n_qubits, 2 * n_qubits):
            qml.RY(weights[l, j], wires=j % n_qubits)

    _expectations = [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]
    return _expectations
    # return qml.expval(qml.PauliZ(0))


class Quanv2d(nn.Module):
    def __init__(self, kernel_size=None, stride=None):
        super(Quanv2d, self).__init__()
        weight_shapes = {"weights": (n_layers, 2 * n_qubits)}
        qnode = qml.QNode(circuit, dev, interface='torch', diff_method='best')
        self.ql1 = qml.qnn.TorchLayer(qnode, weight_shapes)
        self.kernel_size = kernel_size
        self.stride = stride

    def forward(self, X):
        assert len(X.shape) == 4 # (bs, c, w, h)
        bs = X.shape[0]
        XL = []
        for i in range(0, X.shape[2] - 2, stride):
            for j in range(0, X.shape[3] - 2, stride):
                XL.append(self.ql1(torch.flatten(X[:, :, i:i + kernel_size, j:j + kernel_size], start_dim=1)))
        X = torch.cat(XL, dim=1).view(bs,4,6,6)
        return X

class Inception(nn.Module):
    def __init__(self,in_channels):
        super(Inception, self).__init__()

        self.branchClassic_1 = nn.Conv2d(in_channels,4,kernel_size=1,stride=1)
        self.branchClassic_2 = nn.Conv2d(4,8,kernel_size=4,stride=2)

        self.branchQuantum = Quanv2d(kernel_size=4,stride=2)

    def forward(self,x):
        classic = self.branchClassic_1(x)
        classic = self.branchClassic_2(classic)

        quantum = self.branchQuantum(x)

        outputs = [classic,quantum]
        return torch.cat(outputs,dim=1)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.incep = Inception(in_channels=1)
        self.fc1 = nn.Linear(12*6*6,32)
        self.fc2 = nn.Linear(20,10)
        self.lr = nn.LeakyReLU(0.1)

    def forward(self,x):
        bs = x.shape[0]
        x = x.view(bs,1,14,14)
        x = self.incep(x)
        x = self.lr(x)

        x = x.view(bs,-1)
        x = self.lr(self.fc1(x))
        x = self.fc2(x)
        return x

In [8]:
import pennylane as qml
import torch.nn as nn
import torch.nn.functional as F
import torch

torch.manual_seed(0)

n_qubits = 4
n_layers = 1
dev = qml.device('default.qubit', wires=n_qubits)

def circuit(inputs, weights):
    for qub in range(n_qubits):
        qml.Hadamard(wires=qub)
        qml.RY(inputs[qub], wires=qub)
        # qml.RY(inputs[qub], wires=qub)

    for layer in range(n_layers):
        for i in range(n_qubits):
            qml.CRZ(weights[layer,i], wires=[i, (i + 1) % n_qubits])
        for j in range(n_qubits,2*n_qubits):
            qml.RY(weights[layer,j], wires=j % n_qubits)

    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

class Quanv2d(nn.Module):
    def __init__(self, kernel_size):
        super(Quanv2d, self).__init__()
        weight_shapes = {"weights": (n_layers,2*n_qubits)}
        qnode = qml.QNode(circuit, dev, interface='torch', diff_method="best")
        self.ql1 = qml.qnn.TorchLayer(qnode, weight_shapes)
        self.kernel_size = kernel_size
        #self.stride = stride

    def forward(self, x):
        assert len(x.shape) == 4  # (bs, c, w, h)
        bs = x.shape[0]
        # side_len = X.shape[2] - self.kernel_size + 1  # *******
        x_lst = []
        for i in range(0, x.shape[2]-1,2):
            for j in range(0, x.shape[3]-1,2):
                x_lst.append(self.ql1(torch.flatten(x[:, :, i:i + self.kernel_size, j:j + self.kernel_size], start_dim=1)))
        x = torch.cat(x_lst,dim=1).view(bs,4,7,7)
        return x

class Inception(nn.Module):
    def __init__(self,in_channels):
        super(Inception, self).__init__()

        self.branchClassic_1 = nn.Conv2d(in_channels,4,kernel_size=1,stride=1)
        self.branchClassic_2 = nn.Conv2d(4,8,kernel_size=4,stride=2,padding=1)

        self.branchQuantum = Quanv2d(kernel_size=2)

    def forward(self,x):
        classic = self.branchClassic_1(x)
        classic = self.branchClassic_2(classic)

        quantum = self.branchQuantum(x)

        outputs = [classic,quantum]
        return torch.cat(outputs,dim=1)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.incep = Inception(in_channels=1)
        self.fc1 = nn.Linear(12*7*7,32)
        self.fc2 = nn.Linear(32,10)
        self.lr = nn.LeakyReLU(0.1)

    def forward(self,x):
        bs = x.shape[0]
        x = x.view(bs,1,14,14)
        x = self.incep(x)
        x = self.lr(x)

        x = x.view(bs,-1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [9]:
import torch
import torch.nn as nn
import numpy as np
import pennylane as qml
from math import ceil
from math import pi

torch.manual_seed(0)

n_qubits = 4
n_layers = 1
n_class = 3
n_features = 196
image_x_y_dim = 14
kernel_size = n_qubits
stride = 2

dev = qml.device("default.qubit", wires = n_qubits)

def circuit(inputs, weights):
    var_per_qubit = int(len(inputs)/n_qubits) + 1
    encoding_gates = ['RZ', 'RY'] * ceil(var_per_qubit/2)
    for qub in range(n_qubits):
        qml.Hadamard(wires = qub)
        for i in range(var_per_qubit):
            if (qub * var_per_qubit + i) < len(inputs):
                exec('qml.{}({}, wires = {})'.format(encoding_gates[i], inputs[qub * var_per_qubit + i], qub))
            else: #load nothing
                pass

    for l in range(n_layers):
        for i in range(n_qubits):
            qml.CRZ(weights[l, i], wires = [i, (i + 1) % n_qubits])
            #qml.CNOT(wires = [i, (i + 1) % n_qubits])
        for j in range(n_qubits, 2*n_qubits):
            qml.RY(weights[l, j], wires = j % n_qubits)

    _expectations = [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]
    return _expectations
    #return qml.expval(qml.PauliZ(0))

class Quanv2d(nn.Module):
    def __init__(self, kernel_size = None, stride = None):
        super(Quanv2d, self).__init__()
        weight_shapes = {"weights": (n_layers, 2 * n_qubits)}
        qnode = qml.QNode(circuit, dev, interface = 'torch', diff_method = 'best')
        self.ql1 = qml.qnn.TorchLayer(qnode, weight_shapes)
        self.kernel_size = kernel_size
        self.stride = stride

    def forward(self, X):
        assert len(X.shape) == 4 #(bs, c, w, h)
        XL = []
        for i in range(0, X.shape[2]-2, stride):
            for j in range(0, X.shape[3]-2, stride):
                XL.append(self.ql1(torch.flatten(X[:, :, i:i+kernel_size, j:j+kernel_size], start_dim = 1)))
        X = torch.cat(XL, dim = 1)
        return X

class Net(nn.Module):
    # define nn
    def __init__(self):
        super(Net, self).__init__()
        self.ql1 = Quanv2d(kernel_size = kernel_size, stride = stride)

        self.fc1 = nn.Linear(4*6*6, n_class * 2)
        self.lr1 = nn.LeakyReLU(0.1)
        self.fc2 = nn.Linear(n_class * 2, n_class)

    def forward(self, X):

        bs = X.shape[0]
        X = X.view(bs, 1, image_x_y_dim, image_x_y_dim)
        X = self.ql1(X)

        X = self.fc1(X)
        X = self.lr1(X)
        X = self.fc2(X)
        return X

# if __name__ == '__main__':
#     network = Net()
#     random_input = torch.rand(1, n_features)
#     print(network(random_input))
#
#     q1 = Quanv2d(kernel_size = kernel_size, stride = stride)
#     random_input = random_input.view(1, image_x_y_dim, image_x_y_dim)
#     print(q1(random_input))

In [10]:
import torch
import torch.nn as nn
import numpy as np
import pennylane as qml
from math import ceil
from math import pi

torch.manual_seed(0)

n_qubits = 4
n_layers = 1
n_class = 3
n_features = 196
image_x_y_dim = 14
kernel_size = n_qubits
stride = 2

dev = qml.device("default.mixed", wires=n_qubits)


def circuit(inputs, weights):
    var_per_qubit = int(len(inputs) / n_qubits) + 1
    encoding_gates = ['RZ', 'RY'] * ceil(var_per_qubit / 2)
    for qub in range(n_qubits):
        qml.Hadamard(wires=qub)
        for i in range(var_per_qubit):
            if (qub * var_per_qubit + i) < len(inputs):
                exec('qml.{}({}, wires = {})'.format(encoding_gates[i], inputs[qub * var_per_qubit + i], qub))
            else:  # load nothing
                pass

    for l in range(n_layers):
        for i in range(n_qubits):
            qml.CRZ(weights[l, i], wires=[i, (i + 1) % n_qubits])
            # qml.CNOT(wires = [i, (i + 1) % n_qubits])
        for j in range(n_qubits, 2 * n_qubits):
            qml.RY(weights[l, j], wires=j % n_qubits)

    _expectations = [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]
    return _expectations
    # return qml.expval(qml.PauliZ(0))


class Quanv2d(nn.Module):
    def __init__(self, kernel_size=None, stride=None):
        super(Quanv2d, self).__init__()
        weight_shapes = {"weights": (n_layers, 2 * n_qubits)}
        qnode = qml.QNode(circuit, dev, interface='torch', diff_method='best')
        self.ql1 = qml.qnn.TorchLayer(qnode, weight_shapes)
        self.kernel_size = kernel_size
        self.stride = stride

    def forward(self, X):
        assert len(X.shape) == 4  # (bs, c, w, h)
        XL = []
        for i in range(0, X.shape[2] - 2, stride):
            for j in range(0, X.shape[3] - 2, stride):
                XL.append(self.ql1(torch.flatten(X[:, :, i:i + kernel_size, j:j + kernel_size], start_dim=1)))
        X = torch.cat(XL, dim=1)
        return X


class Net(nn.Module):
    # define nn
    def __init__(self):
        super(Net, self).__init__()
        self.ql1 = Quanv2d(kernel_size=kernel_size, stride=stride)

        self.fc1 = nn.Linear(4 * 6 * 6, n_class * 2)
        self.lr1 = nn.LeakyReLU(0.1)
        self.fc2 = nn.Linear(n_class * 2, n_class)

    def forward(self, X):
        bs = X.shape[0]
        X = X.view(bs, 1, image_x_y_dim, image_x_y_dim)
        X = self.ql1(X)

        X = self.fc1(X)
        X = self.lr1(X)
        X = self.fc2(X)
        return X


In [11]:
pip install pennylane tensorflow keras qiskit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 10.5 MB/s eta 0:00:00


In [12]:
import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models


In [13]:
n_qubits = 4
dev = qml.device('default.qubit', wires=n_qubits)

@qml.qnode(dev)
def quantum_circuit(inputs):
    for i in range(n_qubits):
        qml.RX(inputs[i], wires=i)
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]


In [14]:
class QuantumLayer(layers.Layer):
    def __init__(self, n_qubits, **kwargs):
        super(QuantumLayer, self).__init__(**kwargs)
        self.n_qubits = n_qubits

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        outputs = []
        for i in range(batch_size):
            quantum_output = quantum_circuit(inputs[i])
            outputs.append(quantum_output)
        return tf.stack(outputs)


In [21]:
!pip install pennylane tensorflow keras qiskit

import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow import keras
import tensorflow_datasets as tfds


# Load the Brakhis dataset
# Download the dataset from a trusted source
# and replace 'path/to/brakhis/dataset' with the actual path

# Assuming the Brakhis dataset is preprocessed and loaded as NumPy arrays:
# x_train: training images
# y_train: training labels (0 for benign, 1 for malignant)
# x_test: testing images
# y_test: testing labels

# Preprocess data (adjust according to your data format)
# For example:
# x_train = x_train.astype('float32') / 255.0
# x_test = x_test.astype('float32') / 255.0

# Add channel dimension (if necessary)
# x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
# x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)



# Define the quantum circuit
n_qubits = 4
dev = qml.device('default.qubit', wires=n_qubits)

@qml.qnode(dev)
def quantum_circuit(inputs):
    for i in range(n_qubits):
        qml.RX(inputs[i], wires=i)
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# Define the quantum layer
class QuantumLayer(layers.Layer):
    def __init__(self, n_qubits, **kwargs):
        super(QuantumLayer, self).__init__(**kwargs)
        self.n_qubits = n_qubits

    def call(self, inputs):
        # Use tf.map_fn to apply the quantum circuit to each element in the batch
        # This avoids using a Python loop with a symbolic tensor
        quantum_output = tf.map_fn(lambda x: quantum_circuit(x[:self.n_qubits]), inputs)
        return quantum_output

# Define the hybrid QCNN model
def create_hybrid_qcnn(input_shape):
    model = models.Sequential([
        layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        QuantumLayer(n_qubits),
        layers.Dense(1, activation='sigmoid')  # Adjust output units for Brakhis dataset (binary classification)
    ])
    return model

# Create the model
# Assuming x_train.shape[1:] provides the image dimensions
# model = create_hybrid_qcnn(x_train.shape[1:])


# Compile the model
# model.compile(optimizer='adam',
#               loss='binary_crossentropy',  # Use binary_crossentropy for Brakhis dataset
#               metrics=['accuracy'])

# Train the model
# model.fit(x_train